# Análisis alojamientos de AirBnB en Madrid

### **PROBLEMA QUE NOS TRANSMITE EL CLIENTE**
### **“¿Qué puedo hacer para optimizar el precio de mis viviendas?”**
Datos que nos aporta:
- Localización: Barrios de Madrid
- Tipo de vivienda: Apartamento
- Habitaciones: 1-3
- Precio: No definido. Destinado a público en general

In [2]:
# Manipulación y analisis de datos
import pandas as pd
import numpy as np

# Visualización de datos
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

In [3]:
# Importamos los dataframes
airbnb_madrid_conditions = pd.read_csv('DATA/airbnb_madrid_conditions.csv')
airbnb_madrid_host = pd.read_csv('DATA/airbnb_madrid_host.csv')
airbnb_madrid_location = pd.read_csv('DATA/airbnb_madrid_location.csv')
airbnb_madrid_property = pd.read_csv('DATA/airbnb_madrid_property.csv')
airbnb_madrid_reviews = pd.read_csv('DATA/airbnb_madrid_reviews.csv')

In [79]:
# Función para obtener un resumen de cada dataframe
def resumen_dataframe(df, name):
    print(f"Resumen del dataframe: {name}")
    print("Primeras 5 filas:")
    print(df.head())
    print("\nDescripción estadística:")
    print(df.describe(include='all'))
    print("\nInformación del dataframe:")
    print(df.info())
    print("\nValores nulos por columna:")
    print(df.isnull().sum())
    print("\n" + "<>"*50 + "\n")

In [67]:
# Generamos el resumen para cada dataframe para explorarlo mejor
for df, name in zip([airbnb_madrid_conditions, airbnb_madrid_host, airbnb_madrid_location, airbnb_madrid_property, airbnb_madrid_reviews], 
                    ['airbnb_madrid_conditions', 'airbnb_madrid_host', 'airbnb_madrid_location', 'airbnb_madrid_property', 'airbnb_madrid_reviews']):
    resumen_dataframe(df, name)

Resumen del dataframe: airbnb_madrid_conditions
Primeras 5 filas:
   id  price  ...  require_guest_phone_verification  __index_level_0__
0   0   70.0  ...                                 0                  0
1   1   17.0  ...                                 0                  1
2   2   50.0  ...                                 0                  2
3   3   80.0  ...                                 0                  3
4   4  115.0  ...                                 0                  4

[5 rows x 8 columns]

Descripción estadística:
                  id  ...  __index_level_0__
count   21020.000000  ...       21020.000000
unique           NaN  ...                NaN
top              NaN  ...                NaN
freq             NaN  ...                NaN
mean    10644.643911  ...       10644.643911
std      6148.700224  ...        6148.700224
min         0.000000  ...           0.000000
25%      5318.750000  ...        5318.750000
50%     10632.500000  ...       10632.500000
75%     15

In [4]:
# Fusionamos los dataframes
merged_df = airbnb_madrid_conditions.merge(airbnb_madrid_host, how='outer', on='id', suffixes=('_conditions', '_host'))
merged_df = merged_df.merge(airbnb_madrid_location, how='outer', on='id', suffixes=('', '_location'))
merged_df = merged_df.merge(airbnb_madrid_property, how='outer', on='id', suffixes=('', '_property'))
airbnb_df = merged_df.merge(airbnb_madrid_reviews, how='outer', on='id', suffixes=('', '_reviews'))

In [65]:
# Mostramos las columnas del dataframe actualizado para asegurarnos que la fusion se ha realizado correctamente
airbnb_df.columns, airbnb_df.shape

(Index(['id', 'price', 'minimum_nights', 'maximum_nights',
        'cancellation_policy', 'require_guest_profile_picture',
        'require_guest_phone_verification', '__index_level_0___conditions',
        'host_response_time', 'host_response_rate', 'host_is_superhost',
        'host_has_profile_pic', 'host_identity_verified',
        'host_verifications_email', 'host_verifications_phone',
        'host_verifications_reviews', 'host_verifications_facebook',
        'host_verifications_government_id', 'calculated_host_listings_count',
        'host_seniority', '__index_level_0___host', 'neighbourhood',
        'neighbourhood_group', 'latitude', 'longitude', 'dist_km_sol',
        'dist_km_airport', '__index_level_0__', 'property_type', 'room_type',
        'accommodates', 'bathrooms', 'bedrooms', 'beds', 'bed_type',
        'amenities_wifi_internet', 'amenities_tv', 'amenities_air_conditioning',
        'amenities_heating', 'amenities_kitchen', '__index_level_0___property',
        'nu

# Tenemos 21020 viviendas a analizar

# Filtraremos los DF con los datos de las viviendas del cliente: 
Queremos analizar viviendas parecidas para poder encontrar correlaciones mas acordes

In [5]:
# Filtramos el dataframe 'airbnb_df' para incluir solo las filas donde el tipo de propiedad es 'apartment' y el número de dormitorios está entre 1 y 3. Luego, actualizamos 'airbnb_df' con este dataframe filtrado.
filtered_property_types_df = airbnb_df[(airbnb_df['property_type'] == 'apartment') & (airbnb_df['bedrooms'].between(1, 3))]
airbnb_df = filtered_property_types_df
airbnb_df.shape

(15651, 57)

## Es interesante añadir una mascara porque normalmente este tipo de plataformas suele dar un precio/noche. 
 El precio es siempre por noche lo que depende es si se alquila entero o por habitación. Compartidas o privadas

In [6]:
# La lógica de este cambio es la siguiente:
# Primero, creo una máscara 'mask_room' que selecciona las filas donde el tipo de habitación no es 'entire_home_apt' 
# y el número de dormitorios es mayor que 1.
mask_room = (airbnb_df.room_type != 'entire_home_apt') & (airbnb_df.bedrooms > 1)

# Luego, utilizo esta máscara para localizar las filas correspondientes en el dataframe 'airbnb_df' y 
# actualizo la columna 'price' multiplicando el precio actual por el número de dormitorios.
airbnb_df.loc[mask_room, "price"] = airbnb_df.loc[mask_room, "price"] * airbnb_df.loc[mask_room, "bedrooms"]

# Finalmente, muestro el dataframe actualizado.
airbnb_df.shape

(15651, 57)

# Pasamos de 21020 a 15651 viviendas.
**El precio/noche se ha actualizado respecto a las habitaciones**

# HIPÓTESIS 1: Ubicación Privilegiada aumenta el precio
 Objetivo: Determinar si hay una correlación entre la ubicación y el precio
 y recomendar ajustes de precios para propiedades en zonas privilegiadas.
###  VARIABLES a utilizar:
-  price
-  dist_km_sol
-  dist_km_airport
-  neighbourhood_group
-  neighbourhood

In [7]:
# Calculamos el precio mediano por neighbourhood y neighbourhood_group
precio_mediano_neighbourhood = airbnb_df.groupby('neighbourhood')['price'].mean().sort_values(ascending=False)
precio_mediano_neighbourhood_group = airbnb_df.groupby('neighbourhood_group')['price'].mean().sort_values(ascending=False)
precio_mediano_neighbourhood, precio_mediano_neighbourhood_group

(neighbourhood
 San Blas          325.755474
 Usera             226.733032
 Valdeacederas     211.294118
 Cuatro Caminos    209.432099
 Salamanca         181.215054
                      ...    
 Imperial           75.603448
 Carabanchel        72.579060
 Adelfas            67.766667
 Estrella           67.214286
 Castilla           56.368421
 Name: price, Length: 66, dtype: float64,
 neighbourhood_group
 San Blas - Canillejas    328.802920
 Usera                    226.733032
 Salamanca                153.796657
 Tetuán                   153.143084
 Barajas                  152.594059
 Hortaleza                128.033149
 Vicálvaro                124.111111
 Chamberí                 121.292909
 Centro                   118.055711
 Moncloa - Aravaca        113.067442
 Latina                   110.307692
 Chamartín                108.956835
 Retiro                   104.078095
 Ciudad Lineal            102.997859
 Villa de Vallecas        101.101695
 Arganzuela                96.655026


In [8]:
# Calculamos el precio medio por neighbourhood_group
precio_medio_neighbourhood_group = airbnb_df.groupby('neighbourhood_group')['price'].mean()


In [9]:
# Ordenamos los neighbourhood por precio promedio de menor a mayor
precio_mediano_neighbourhood_group_sorted = precio_mediano_neighbourhood.sort_values(ascending=True).index.tolist()

# Creamos un boxplot con Plotly Express para tener una mejor visualización de los datos
boxplot_pm_neighbourhood_group = px.box(airbnb_df, x='neighbourhood', y='price', color='neighbourhood', 
             category_orders={'neighbourhood': precio_mediano_neighbourhood_group_sorted},
             title='Boxplot de los precios medianos por Grupo de barrio')

# Mejoramos la legibilidad de las etiquetas
boxplot_pm_neighbourhood_group.update_layout(
    xaxis_title='Neighbourhood',
    yaxis_title='Price',
    title_font_size=20,
    xaxis_tickangle=-45,
    xaxis_tickfont_size=10,
    yaxis_tickfont_size=12,
    showlegend=False
)

# Mostrar el gráfico
boxplot_pm_neighbourhood_group.show()

### No parece una buena opción pintar un grafico con tantas variables, es dificil de entender. 
### Usaremos la variable neighbourhood_group

In [10]:
# Ordenamos los neighbourhood_groups por precio promedio de menor a mayor
precio_medio_neighbourhood_group_sorted = precio_medio_neighbourhood_group.sort_values(ascending=True).index.tolist()

# Creamos un boxplot con Plotly Express para tener una mejor visualización de los datos
boxplot_pm_neighbourhood_group = px.box(airbnb_df, x='neighbourhood_group', y='price', color='neighbourhood_group', 
             category_orders={'neighbourhood_group': precio_medio_neighbourhood_group_sorted},
             title='Boxplot de los precios medianos por Grupo de barrio')

# Mejoramos la legibilidad de las etiquetas
boxplot_pm_neighbourhood_group.update_layout(
    xaxis_title='Neighbourhood Group',
    yaxis_title='Price',
    title_font_size=20,
    xaxis_tickangle=-45,
    xaxis_tickfont_size=10,
    yaxis_tickfont_size=12,
    showlegend=False
)

# Mostrar el gráfico
boxplot_pm_neighbourhood_group.show()

### Aun centrándonos en solo apartamentos con max 1-3 habitaciones los precios siguen llegando a valores muy altos que desvirtúan los gráficos, pero si nos fijamos son la minoría. Menos del 10% de las viviendas tienen precios atípicos. 

# Tratamos los outliers
El cliente no ha especificado el precio pero si ha definido ques vivienda común. Por eso centraremos el análisis en el 90% de las viviendas que son las que tienen un precio < 200 €
# Como estamos usando Plotly se tendrá que hacer zoom al rango 200€. 

In [11]:
# Filtraremos los 10 neighbourhood_group más caros por precio medio y los ordenaremos de más barato a más caro
top_10_expensive_neighbourhoods_groups = airbnb_df.groupby('neighbourhood_group')['price'].mean().nlargest(10).sort_values(ascending=True)

# Filtraremos el DataFrame original para obtener solo los 10 neighbourhood_group más caros
top_10_expensive_neighbourhoods_df = airbnb_df[airbnb_df['neighbourhood_group'].isin(top_10_expensive_neighbourhoods_groups.index)]

In [14]:
# Crearemos un gráfico de barras con Plotly Express usando df que hemos creado
barplot_top_10_expensive_neighbourhoods = px.bar(
    top_10_expensive_neighbourhoods_groups,
    x=top_10_expensive_neighbourhoods_groups.index,
    y='price',
    title='Top 10 Grupo de barrios más caros por precio medio',
    labels={'neighbourhood_group': 'Neighbourhood Group', 'price': 'Average Price'}
)

# Añadimos etiquetas de valor en las barras
barplot_top_10_expensive_neighbourhoods.update_traces(
    texttemplate='%{y:.2f}€',
    textposition='inside',
    textfont_size=12
)

# Mejoraramos la legibilidad de las etiquetas
barplot_top_10_expensive_neighbourhoods.update_layout(
    xaxis_title='Neighbourhood Group',
    yaxis_title='Price',
    title_font_size=20,
    xaxis_tickangle=-45,
    xaxis_tickfont_size=10,
    yaxis_tickfont_size=12
)

# Calculamos el precio medio
precio_medio = precio_medio_neighbourhood_group.mean()

# Añadimos una línea de puntitos roja que marque el precio medio
barplot_top_10_expensive_neighbourhoods.add_shape(
    type='line',
    x0=-0.5,
    x1=len(top_10_expensive_neighbourhoods_groups) - 0.5,
    y0=precio_medio,
    y1=precio_medio,
    line=dict(color='red', width=2, dash='dot')
)

# Añadimos una anotación para la línea de precio medio
barplot_top_10_expensive_neighbourhoods.add_annotation(
    x=len(top_10_expensive_neighbourhoods_groups) - 0.5,
    y=precio_medio,
    text=f'Precio medio: {precio_medio:.2f}€',
    showarrow=False,
    font=dict(color='red', size=12),
    xanchor='left'
)

# Mostramos el gráfico
barplot_top_10_expensive_neighbourhoods.show()

## Solo San Blas y Usera tienen un precio promedio que supere claramente los 200€

In [15]:
# Contar cuántos pisos hay en cada grupo de barrio
neighbourhood_group_counts = airbnb_df['neighbourhood_group'].value_counts()

# Crear un DataFrame con los datos necesarios para el gráfico
bubble_data = pd.DataFrame({
    'neighbourhood_group': neighbourhood_group_counts.index,
    'count': neighbourhood_group_counts.values,
    'price': precio_medio_neighbourhood_group[neighbourhood_group_counts.index].values
})

# Ordenar el DataFrame por la columna 'count' de menor a mayor
bubble_data = bubble_data.sort_values(by='count')

# Crear un gráfico de burbujas con Plotly
bubble_chart = px.scatter(
    bubble_data,
    x='neighbourhood_group',
    y='price',
    size='count',
    color='price',
    title='Precio Medio y Cantidad de Pisos por Grupo de Barrio',
    labels={'neighbourhood_group': 'Grupo de Barrio', 'price': 'Precio Medio', 'count': 'Cantidad de Pisos'},
    size_max=60,
    color_continuous_scale=px.colors.sequential.Viridis
)

# Mejorar la legibilidad de las etiquetas
bubble_chart.update_layout(
    title_font_size=20,
    xaxis_title='neighbourhood_group',
    yaxis_title='Price',
    xaxis_tickangle=-45,
    xaxis_tickfont_size=10,
    yaxis_tickfont_size=12,
    showlegend=True
)

# Mostrar el gráfico
bubble_chart.show()

## La inmesa mayoria de los pisos estan concentrados en CENTRO. 
Podemos observar mejor la distribución dentro del rango de los 200€

In [16]:
# Creamos el gráfico de dispersión con Plotly
scatter_plot = px.scatter(
    airbnb_df,
    x="dist_km_sol",
    y="price",
    color="neighbourhood_group",
    title='Gráfico de Dispersión del Precio por Distancia a Sol con Grupo de Barrio',
    labels={'dist_km_sol', 'price'},
    color_discrete_sequence=px.colors.qualitative.Plotly,  # Cambiado de Husl a Plotly
    opacity=0.6
)

# Mejoramos la legibilidad de las etiquetas
scatter_plot.update_layout(
    title_font_size=16,
    xaxis_title_font_size=14,
    yaxis_title_font_size=14,
    xaxis_tickfont_size=12,
    yaxis_tickfont_size=12,
    legend_title_font_size=16,  # Aumentar el tamaño de la fuente del título de la leyenda
    legend_font_size=14,  # Aumentar el tamaño de la fuente de la leyenda
    legend=dict(title='Grupo de Barrio', x=1.15, y=1),
    xaxis=dict(tickmode='linear', tick0=0, dtick=1),  # Configurar los ticks del eje x
    margin=dict(l=0, r=0, t=50, b=0)  # Ajustar los márgenes
)

# Añadiremos una cuadrícula para mejorar la legibilidad
scatter_plot.update_xaxes(showgrid=True, gridwidth=0.5, gridcolor='LightGrey', tickangle=-45)
scatter_plot.update_yaxes(showgrid=True, gridwidth=0.5, gridcolor='LightGrey')

# Mostramos el gráfico
scatter_plot.show()

In [17]:
# Creamos el gráfico de dispersión con Plotly para los top 10 barrios
scatter_plot_top10 = px.scatter(
    top_10_expensive_neighbourhoods_df,
    x="dist_km_sol",
    y="price",
    color="neighbourhood_group",
    title='Gráfico de Dispersión del Precio por Distancia a Sol con Top 10 Barrios',
    labels={'dist_km_sol', 'price'},
    color_discrete_sequence=px.colors.qualitative.Plotly,  # Cambiado de Husl a Plotly
    opacity=0.6
)

# Mejoramos la legibilidad de las etiquetas
scatter_plot_top10.update_layout(
    title_font_size=16,
    xaxis_title_font_size=14,
    yaxis_title_font_size=14,
    xaxis_tickfont_size=12,
    yaxis_tickfont_size=12,
    legend_title_font_size=16,  # Aumentar el tamaño de la fuente del título de la leyenda
    legend_font_size=14,  # Aumentar el tamaño de la fuente de la leyenda
    legend=dict(title='Grupo de Barrio', x=1.15, y=1),
    xaxis=dict(tickmode='linear', tick0=0, dtick=1),  # Configurar los ticks del eje x
    margin=dict(l=0, r=0, t=50, b=0)  # Ajustar los márgenes
)

# Añadiremos una cuadrícula para mejorar la legibilidad
scatter_plot_top10.update_xaxes(showgrid=True, gridwidth=0.5, gridcolor='LightGrey', tickangle=-45)
scatter_plot_top10.update_yaxes(showgrid=True, gridwidth=0.5, gridcolor='LightGrey')

# Mostramos el gráfico
scatter_plot_top10.show()

## Podemos observar una clara correlación entre la cercania a Sol y los precios. 
 Contra mas cerca mayor precio podemos pedir por nuestra vivienda. Pasara lo mismo con la distancia al aeropuerto?

In [18]:
# Creamos el gráfico de dispersión con Plotly
scatter_plot = px.scatter(
    airbnb_df,
    x="dist_km_airport",
    y="price",
    color="neighbourhood_group",
    title='Gráfico de Dispersión del Precio por Distancia al Aeropuerto con Grupo de Barrio',
    labels={'dist_km_airport': 'Distancia al Aeropuerto (km)', 'price': 'Precio (€)'},
    color_discrete_sequence=px.colors.qualitative.Plotly,  # Cambiado de Husl a Plotly
    opacity=0.6
)

# Mejoramos la legibilidad de las etiquetas
scatter_plot.update_layout(
    title_font_size=16,
    xaxis_title_font_size=14,
    yaxis_title_font_size=14,
    xaxis_tickfont_size=12,
    yaxis_tickfont_size=12,
    legend_title_font_size=16,  # Aumentar el tamaño de la fuente del título de la leyenda
    legend_font_size=14,  # Aumentar el tamaño de la fuente de la leyenda
    legend=dict(title='Grupo de Barrio', x=1.15, y=1),
    xaxis=dict(tickmode='linear', tick0=0, dtick=1),  # Configurar los ticks del eje x
    margin=dict(l=0, r=0, t=50, b=0)  # Ajustar los márgenes
)

# Añadiremos una cuadrícula para mejorar la legibilidad
scatter_plot.update_xaxes(showgrid=True, gridwidth=0.5, gridcolor='LightGrey', tickangle=-45)
scatter_plot.update_yaxes(showgrid=True, gridwidth=0.5, gridcolor='LightGrey')

# Mostramos el gráfico
scatter_plot.show()

## No se aprecia una tendencia positiva. De hecho si buscamos a cuantos km del aeropuerto esta el centro nos daremos cuenta de un pequeño detalle. 
# 13km, exactamente. 
Este grafico nos confirma la tendencia a mayores precio en el centro.

In [19]:
# Creamos el gráfico de dispersión con Plotly para el top 10
scatter_plot = px.scatter(
    top_10_expensive_neighbourhoods_df,
    x="dist_km_airport",
    y="price",
    color="neighbourhood_group",
    title='Gráfico de Dispersión del Precio por Distancia al Aeropuerto con Top 10 Grupos de Barrio',
    labels={'dist_km_airport': 'Distancia al Aeropuerto (km)', 'price': 'Precio (€)'},
    color_discrete_sequence=px.colors.qualitative.Plotly,  # Cambiado de Husl a Plotly
    opacity=0.6
)

# Mejoramos la legibilidad de las etiquetas
scatter_plot.update_layout(
    title_font_size=16,
    xaxis_title_font_size=14,
    yaxis_title_font_size=14,
    xaxis_tickfont_size=12,
    yaxis_tickfont_size=12,
    legend_title_font_size=16,  # Aumentar el tamaño de la fuente del título de la leyenda
    legend_font_size=14,  # Aumentar el tamaño de la fuente de la leyenda
    legend=dict(title='Grupo de Barrio', x=1.15, y=1),
    xaxis=dict(tickmode='linear', tick0=0, dtick=1),  # Configurar los ticks del eje x
    margin=dict(l=0, r=0, t=50, b=0)  # Ajustar los márgenes
)

# Añadiremos una cuadrícula para mejorar la legibilidad
scatter_plot.update_xaxes(showgrid=True, gridwidth=0.5, gridcolor='LightGrey', tickangle=-45)
scatter_plot.update_yaxes(showgrid=True, gridwidth=0.5, gridcolor='LightGrey')

# Mostramos el gráfico
scatter_plot.show()

## Mismos resultados, se cumple la tendencia.

# CONCLUSIONES HIPOTESIS 1:

## Equilibrar el precio a la media del barrio

## Contra más cerca del centro más capacidad de incrementar el precio

## San Blas y Usera tienen mayor capacidad de subir precios por encima de la media de Madrid

# HIPÓTESIS 2: Impacto de Reseñas y Superhosts en el Precio
Objetivo: Evaluar cómo las reseñas y el estatus de superhost afectan el precio de las propiedades.
###  VARIABLES a utilizar:
-  price
-  review_scores_rating
-  number_of_reviews
-  neighbourhood_group
-  host_is_superhost

In [20]:
# Creamos un gráfico de dispersión para ver la relación entre precio y review_scores_rating
scatterplot_Precio_reseña = px.scatter(
    airbnb_df, 
    x='review_scores_rating', 
    y='price', 
    color='host_is_superhost',
    title='Precio vs Puntuación de Reseñas y si es SuperHost',
    labels={'review_scores_rating': 'Puntuación de Reseñas', 'price': 'Precio'},
    hover_data=['neighbourhood_group']
)

# Mejoramos la legibilidad del gráfico
scatterplot_Precio_reseña.update_layout(
    title_font_size=20,
    xaxis_title_font_size=16,
    yaxis_title_font_size=16,
    legend_title_font_size=14,
    legend_font_size=12,
    hoverlabel_font_size=12
)

# Mostramos el gráfico
scatterplot_Precio_reseña.show()

## Vemos reflejada una clara tendencia al alza en la puntuación de las reseñas contra el precio. El amarillo del SuperHost se ve distribuido con fuerza en la zona de las máximas puntuaciones, pero es homogéneo en precios bajos como altos.

Podemos concluir que las puntuaciones son extremadamente importantes para el precio. En cambio, el super host no parece afectar el precio de una manera contundente.

In [21]:
# Creamos gráfico de dispersión para ver la relación entre precio y número de reseñas
scatter_nreseñas_precio = px.scatter(
    airbnb_df, 
    x='number_of_reviews', 
    y='price', 
    color='host_is_superhost',
    title='Precio vs Número de Reseñas',
    labels={'number_of_reviews': 'Número de Reseñas', 'price': 'Precio'},
    hover_data=['neighbourhood_group']
)

# Actualizamos el diseño del gráfico para mejorar la legibilidad
scatter_nreseñas_precio.update_layout(
    title_font_size=20,
    xaxis_title_font_size=16,
    yaxis_title_font_size=16,
    legend_title_font_size=14,
    legend_font_size=12,
    hoverlabel_font_size=12
)

# Mejoramos la legibilidad de las etiquetas del eje x
scatter_nreseñas_precio.update_xaxes(tickangle=-45)

# Mostramos el gráfico
scatter_nreseñas_precio.show()

El número de reseñas tiene una alta concentración en las 200 primeras. Pero el impacto en el precio se concentra en las primeras 100, a partir de ahí podemos apreciar un descenso. A su vez vemos de nuevo totalmente distribuido el Superhost. 

In [22]:
# Creamos gráfico de dispersión para comparar los precios en diferentes grupos de vecindarios
boxplot_neighbourhood_group_super = px.box(
    airbnb_df, 
    x='neighbourhood_group', 
    y='price', 
    color='host_is_superhost',
    title='Distribución de Precios por Grupo de Barrios y Estado de Superhost',
    labels={'neighbourhood_group': 'Grupo de Barrios', 'price': 'Precio'}
)

# Actualizamos el diseño del gráfico para ordenar las categorías del eje x de forma descendente y añadimos título a la leyenda
boxplot_neighbourhood_group_super.update_layout(
    xaxis={'categoryorder': 'total descending'},
    legend_title_text='Superhost',
    title_font_size=20,
    xaxis_title_font_size=16,
    yaxis_title_font_size=16,
    legend_title_font_size=14,
    legend_font_size=12,
    hoverlabel_font_size=12
)

# Mejoramos la legibilidad de las etiquetas del eje x
boxplot_neighbourhood_group_super.update_xaxes(tickangle=-45)

# Mostramos el gráfico
boxplot_neighbourhood_group_super.show()

Para poder estudiar mejor el impacto del SuperHost lo comparamos con los barrios.
En la gran mayoría, el precio medio es más alto en los que no son SuperHost.
Excepto en Centro, Chamberi y Moncloa

In [23]:
# Creamos un boxplot para comparar los precios en diferentes grupos de vecindarios
boxplot_t10_super = px.box(
    top_10_expensive_neighbourhoods_df, 
    x='neighbourhood_group', 
    y='price', 
    color='host_is_superhost',
    title='Distribución de Precios del Top 10 Barrios y Estado de Superhost',
    labels={'neighbourhood_group': 'Top 10 de Barrios', 'price': 'Precio'}
)

# Actualizamos el diseño del gráfico para ordenar las categorías del eje x de forma descendente y añadimos título a la leyenda
boxplot_t10_super.update_layout(
    xaxis={'categoryorder': 'total descending'},
    legend_title_text='Superhost',
    title_font_size=20,
    xaxis_title_font_size=16,
    yaxis_title_font_size=16,
    legend_title_font_size=14,
    legend_font_size=12,
    hoverlabel_font_size=12
)

# Mejoramos la legibilidad de las etiquetas del eje x
boxplot_t10_super.update_xaxes(tickangle=-45)

# Mostramos el gráfico
boxplot_t10_super.show()

Es curioso cómo el impacto de no ser SuperHost en este caso es bastante significativo. Parece que las zonas más centrales sí que pelean al SuperHost, debido a su alto nivel de competencia. En esos casos, sí es diferenciador el SuperHost.

# CONCLUSIONES HIPOTESIS 2:

## Conseguir buenas reseñas es altamente recomendable

## Cantidad de reseñas para incrementar la confianza y poder aumentar el precio

## El SuperHost es importante para diferenciarse en zonas céntricas por la alta cantidad de viviendas

# HIPÓTESIS 3: Servicios aumentan el Precio
Objetivo: Determinar si la oferta de servicios como Wi-Fi, aire acondicionado, cocina y TV influye en el precio.
###  VARIABLES a utilizar:
-  price
-  neighbourhood_group
-  amenities_wifi_internet
-  amenities_tv
-  amenities_air_conditioning
-  amenities_kitchen

In [24]:
# Filtramos las propiedades por la presencia de servicios en airbnb_df

# Filtramos por Wi-Fi
wifi_properties = airbnb_df[airbnb_df['amenities_wifi_internet'] == True]

# Filtramos por TV
tv_properties = airbnb_df[airbnb_df['amenities_tv'] == True]

# Filtramos por aire acondicionado
ac_properties = airbnb_df[airbnb_df['amenities_air_conditioning'] == True]

# Filtramos por cocina
kitchen_properties = airbnb_df[airbnb_df['amenities_kitchen'] == True]

In [25]:
# Calculamos el precio medio de propiedades con y sin cada servicio

# Precio medio con y sin Wi-Fi
mean_price_with_wifi = wifi_properties['price'].mean()
mean_price_without_wifi = airbnb_df[airbnb_df['amenities_wifi_internet'] == False]['price'].mean()

# Precio medio con y sin TV
mean_price_with_tv = tv_properties['price'].mean()
mean_price_without_tv = airbnb_df[airbnb_df['amenities_tv'] == False]['price'].mean()

# Precio medio con y sin aire acondicionado
mean_price_with_ac = ac_properties['price'].mean()
mean_price_without_ac = airbnb_df[airbnb_df['amenities_air_conditioning'] == False]['price'].mean()

# Precio medio con y sin cocina
mean_price_with_kitchen = kitchen_properties['price'].mean()
mean_price_without_kitchen = airbnb_df[airbnb_df['amenities_kitchen'] == False]['price'].mean()

# Resultados
mean_prices = {
    'Wi-Fi': {'Con': mean_price_with_wifi, 'Sin': mean_price_without_wifi},
    'TV': {'Con': mean_price_with_tv, 'Sin': mean_price_without_tv},
    'Aire Acondicionado': {'Con': mean_price_with_ac, 'Sin': mean_price_without_ac},
    'Cocina': {'Con': mean_price_with_kitchen, 'Sin': mean_price_without_kitchen}
}

mean_prices

{'Wi-Fi': {'Con': 120.42066986922872, 'Sin': 155.42232277526395},
 'TV': {'Con': 122.88930990003225, 'Sin': 118.1370495842316},
 'Aire Acondicionado': {'Con': 133.62393681652492, 'Sin': 101.85974025974026},
 'Cocina': {'Con': 123.07761545385308, 'Sin': 107.86423841059603}}

In [26]:
# Creamos listas para los servicios y sus precios medios
services = ['Wi-Fi', 'TV', 'Aire Acondicionado', 'Cocina']
mean_prices_with = [mean_prices[service]['Con'] for service in services]
mean_prices_without = [mean_prices[service]['Sin'] for service in services]

# Creamos un DataFrame para plotly express
data_servicios = {
    'Servicios': services * 2,
    'Precio Medio': mean_prices_with + mean_prices_without,
    'Tipo': ['Con Servicio'] * len(services) + ['Sin Servicio'] * len(services)
}

servicios_df = pd.DataFrame(data_servicios)

# Creamos el gráfico de barras con plotly express
bar_servicios = px.bar(
    servicios_df, 
    x='Servicios', 
    y='Precio Medio', 
    color='Tipo', 
    barmode='group',
    title='Comparación de Precios Medios con y sin Servicios'
)

# Mostramos el gráfico
bar_servicios.show()

## A excepción del Wifi existe una relación positiva en tener un servicio o no en la vivienda para poder justificar un mayor precio.

In [27]:
# Datos de precios medios por barrio
wifi_prices_by_neighbourhood = (
    airbnb_df[airbnb_df['amenities_wifi_internet'] == True]
    .groupby('neighbourhood_group')['price']
    .mean()
    .reset_index()
)

no_wifi_prices_by_neighbourhood = (
    airbnb_df[airbnb_df['amenities_wifi_internet'] == False]
    .groupby('neighbourhood_group')['price']
    .mean()
    .reset_index()
)

# Renombramos columnas para claridad
wifi_prices_by_neighbourhood.columns = ['neighbourhood_group', 'price_with_wifi']
no_wifi_prices_by_neighbourhood.columns = ['neighbourhood_group', 'price_without_wifi']

# Unimos los datos en un solo DataFrame
merged_prices = wifi_prices_by_neighbourhood.merge(no_wifi_prices_by_neighbourhood, on='neighbourhood_group')

# Creamos gráfico de barras con Plotly Express
wifi_bar = px.bar(
    merged_prices, 
    x='neighbourhood_group', 
    y=['price_with_wifi', 'price_without_wifi'], 
    barmode='group', 
    labels={'value': 'Precio Medio', 'neighbourhood_group': 'Barrios'},
    title='Comparación del Precio Medio de Propiedades con y sin Wi-Fi por Barrio'
)
# Mostramos el gráfico
wifi_bar.update_layout(xaxis_tickangle=-45)
wifi_bar.show()

Efectivamente parece que la variable del Wi-fi no parece ser importante. Podriamos ahorrar en gastos.

In [28]:
# Datos de precios medios por barrio
tv_prices_by_neighbourhood = (
    airbnb_df[airbnb_df['amenities_tv'] == True]
    .groupby('neighbourhood_group')['price']
    .mean()
    .reset_index()
)

no_tv_prices_by_neighbourhood = (
    airbnb_df[airbnb_df['amenities_tv'] == False]
    .groupby('neighbourhood_group')['price']
    .mean()
    .reset_index()
)

# Renombramos columnas para claridad
tv_prices_by_neighbourhood.columns = ['neighbourhood_group', 'price_with_tv']
no_tv_prices_by_neighbourhood.columns = ['neighbourhood_group', 'price_without_tv']

# Unimos los datos en un solo DataFrame
merged_prices = tv_prices_by_neighbourhood.merge(no_tv_prices_by_neighbourhood, on='neighbourhood_group')

# Creamos gráfico de barras con Plotly Express
tv_bar = px.bar(
    merged_prices, 
    x='neighbourhood_group', 
    y=['price_with_tv', 'price_without_tv'], 
    barmode='group', 
    labels={'value': 'Precio Medio', 'neighbourhood_group': 'Barrios'},
    title='Comparación del Precio Medio de Propiedades con y sin TV por Barrio'
)
# Mostramos el gráfico
tv_bar.update_layout(xaxis_tickangle=-45)
tv_bar.show()

Existe una pelea intensa con la variable TV. Hoy en día, con la tecnología de los móviles y la naturaleza del airbnb de ser para turismo, no parece que sea representativa.

In [29]:
# Datos de precios medios por barrio
ac_prices_by_neighbourhood = (
    airbnb_df[airbnb_df['amenities_air_conditioning'] == True]
    .groupby('neighbourhood_group')['price']
    .mean()
    .reset_index()
)

no_ac_prices_by_neighbourhood = (
    airbnb_df[airbnb_df['amenities_air_conditioning'] == False]
    .groupby('neighbourhood_group')['price']
    .mean()
    .reset_index()
)

# Renombrar columnas para claridad
ac_prices_by_neighbourhood.columns = ['neighbourhood_group', 'price_with_ac']
no_ac_prices_by_neighbourhood.columns = ['neighbourhood_group', 'price_without_ac']

# Unir los datos en un solo DataFrame
merged_prices = ac_prices_by_neighbourhood.merge(no_ac_prices_by_neighbourhood, on='neighbourhood_group')

# Crear gráfico de barras con Plotly Express
ac_bar = px.bar(
    merged_prices, 
    x='neighbourhood_group', 
    y=['price_with_ac', 'price_without_ac'], 
    barmode='group', 
    labels={'value': 'Precio Medio', 'neighbourhood_group': 'Barrios'},
    title='Comparación del Precio Medio de Propiedades con y sin Aire Acondicionado por Barrio'
)
# Mostramos el gráfico
ac_bar.update_layout(xaxis_tickangle=-45)
ac_bar.show()

Con el aire acondicionado ya vemos que sí hay un vencedor. El aire es necesario en el airbnb para justificar mayores precios.


In [30]:
# Datos de precios medios por barrio
kitchen_prices_by_neighbourhood = (
    airbnb_df[airbnb_df['amenities_kitchen'] == True]
    .groupby('neighbourhood_group')['price']
    .mean()
    .reset_index()
)

no_kitchen_prices_by_neighbourhood = (
    airbnb_df[airbnb_df['amenities_kitchen'] == False]
    .groupby('neighbourhood_group')['price']
    .mean()
    .reset_index()
)

# Renombrar columnas para claridad
kitchen_prices_by_neighbourhood.columns = ['neighbourhood_group', 'price_with_kitchen']
no_kitchen_prices_by_neighbourhood.columns = ['neighbourhood_group', 'price_without_kitchen']

# Unir los datos en un solo DataFrame
merged_prices = kitchen_prices_by_neighbourhood.merge(no_kitchen_prices_by_neighbourhood, on='neighbourhood_group')

# Crear gráfico de barras con Plotly Express
kit_bar = px.bar(
    merged_prices, 
    x='neighbourhood_group', 
    y=['price_with_kitchen', 'price_without_kitchen'], 
    barmode='group', 
    labels={'value': 'Precio Medio', 'neighbourhood_group': 'Barrios'},
    title='Comparación del Precio Medio de Propiedades con y sin Cocina por Barrio'
)
# Mostramos el gráfico
kit_bar.update_layout(xaxis_tickangle=-45)
kit_bar.show()

Otro claro vencedor. Poder cocinar en el airbnb permite ahorrar a los huéspedes.

A excepción de barrios más ricos, la cocina parece una variable necesaria también a la hora de subir el precio.

# CONCLUSIONES HIPOTESIS 3:

## El wi-fi no parece ser importante. Podríamos ahorrar en gastos

## El aire es necesario en el airbnb para justificar mayores precios

## La cocina también es diferenciadora a la hora de poder subir precios